In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from dotenv import load_dotenv
# lädt die (versteckte) Datei die im selben Ordner liegt wie dieses notebook file
# die Datei muss .env benannt sein, diese Dateien sind in der Regel ausgeblendet, 
# weil Dateien mit . andeuten, dass diese im Explorer ausgeblendet werden.
# In der Datei
load_dotenv()

## Header
Wo wir unseren geheimen Schlüssel für ChatGPT mitschicken mit dem wir uns automatisch einloggen und authentifizieren (und auch unser Budget ansprechen)

In [ ]:
import os
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.environ["API_KEY"]}",
}

Die Daten die wir mit unsere Anfrage schicken - dies entspricht der GESAMTEN Unterhaltung!

# Tabellen interpretieren

In [ ]:
import base64
from IPython.display import Image, display

# Tabelle aus https://www.statistischebibliothek.de/mir/servlets/MCRFileNodeServlet/DEAusgabe_derivate_00000632/Wirtschaft_und_Statistik-1980-09.pdf
# dokument von 1980
with open("tabelle.png", mode='rb') as h:
    image_data = h.read()

display(Image(data=image_data))

In [ ]:
base64_encoded = base64.b64encode(image_data).decode('utf-8')

image_payload = {
  # Das 4o klappt in der Regel besser für Bildanalysen
  "model": "gpt-4o",
  "messages": [
    {"role": "system", 
     "content": "Du bist ein Experte darin PDF Tabellen zu analysieren"
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": """Wie viele Arbeitsstunden wurden 1980 in Nordrhein-Westfalen geleistet ?"""
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/png;base64,{base64_encoded}"
          }
        }         
      ]
    }
  ],
  "temperature": 0.5
}

In [ ]:
import pprint
import requests

def call_openai(payload: dict) -> dict:
    ENDPOINT = "https://api.openai.com/v1/chat/completions"
        
    response = requests.post(ENDPOINT, headers=headers, json=image_payload)
    if response.status_code != 200:
        logger.warning(f"Fehlerhafte Antwort: {response.text}")
    endpoint_response = response.json()
    return endpoint_response

endpoint_response = call_openai(payload=image_payload)

In [ ]:
chat_answer = endpoint_response.get('choices', [])[0].get('message', {}).get('content', "FEHLER")
chat_answer

# Extraktion von Tabellen

In [ ]:
schema = {
    "type": "function",
    "function": {
        "name": "tabellen_extraktion",
        "description": "Extrahiere Schlüsselinformationen aus Tabellen",
        "parameters": {
            "type": "object",
            "properties": {   
                "tabellen_zeile": {
                    "type": "array",  
                    "description": "Die Zeilen der Tabelle",
                    "items": {
                        "type": "object",
                        "properties": {
                            "bundesland": {
                                "type": "string",
                                "description": "Der Name des Bundesland"
                            },
                            "anzahl_beschaeftigte_1979": {
                                "type": "number",
                                "description": "Anzahl der Beschäftigten in 1979"
                            },
                            "anzahl_beschaeftigte_1980": {
                                "type": "number",
                                "description": "Anzahl der Beschäftigten in 1980"
                            }, 
                            "veraenderung_der_anzahl_beschaeftigte": {
                                "type": "number",
                                "description": "Veränderung der Beschäftigten zum Vorjahres Zeitraum 1979"
                            },    
                            "arbeitsstunden_1979": {
                                "type": "string",
                                "description": "Anzahl der geleisteten Arbeitsstunden in 1979"
                            },
                            "arbeitsstunden_1980": {
                                "type": "string",
                                "description": "Anzahl der geleisteten Arbeitsstunden in 1980"
                            },    
                            "veraenderung_der_anzahl_geleisteter_arbeitsstunden": {
                                "type": "number",
                                "description": "Veränderung der Beschäftigten zum Vorjahres Zeitraum 1979"
                            },                             
                        }
                    }
                }                
            }
        }
    }
}

In [ ]:
image_payload = {
  # Das 4o klappt in der Regel besser für Bildanalysen
  "model": "gpt-4o",
  "messages": [
    {"role": "system", 
     "content": "Du bist ein Experte darin PDF Tabellen zu analysieren"
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": """Extrahiere die Zeilen der Tabelle"""
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/png;base64,{base64_encoded}"
          }
        }         
      ]
    }
  ],
  "tools": [schema],
  "tool_choice": "required",   
  "temperature": 0.5
}

In [ ]:
endpoint_response = call_openai(payload=image_payload)

## Konvertieren des Extraktionsergebnis als Tabelle in einem Pandas Dataframe

In [ ]:
import json
import pandas as pd
data_frames = []
for tool_call_result in endpoint_response.get('choices', [])[0].get('message', {}).get('tool_calls', []):
    data = json.loads(tool_call_result.get('function', {}).get('arguments'))
    frame = data.get("tabellen_zeile", [])
    data_frames.append(pd.DataFrame(frame))

data_frames[0]

# Komplexe Tabellen mit kombinierten Zellen

In [ ]:
import base64
from IPython.display import Image, display

with open("komplexe_zellen.png", mode='rb') as h:
    image_data = h.read()

display(Image(data=image_data))

In [ ]:
base64_encoded = base64.b64encode(image_data).decode('utf-8')

image_payload = {
  # Das 4o klappt in der Regel besser für Bildanalysen
  "model": "gpt-4o",
  "messages": [
    {"role": "system", 
     "content": "Du bist ein Experte darin PDF Tabellen zu analysieren"
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": """Welcher Wert steht für die Zeile "Wert C" in 2018?"""
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/png;base64,{base64_encoded}"
          }
        }         
      ]
    }
  ],
  "temperature": 0.5
}

In [ ]:
endpoint_response = call_openai(payload=image_payload)

In [ ]:
# Komplexe Tabellen Strukture fangen bereits bei kombinierten Zellen an und die Extraktion neight dazu an solchen Stellen Fehler zu produzieren.
# Als Daumen-Regel gilt, primitive, einfache Tabellen die zeilenweise organisiert sind, sind in der Regel gut zu extrahieren.
# Sobald 'kosmetische' Aufhübschung vorhanden ist, wird die automatische Interpretation von Tabellen herausforderned.
endpoint_response.get('choices')[0].get('message', {}).get('content')

# Analyse von komplexen Grafiken / Liniendiagrammen

In [ ]:
import base64
from IPython.display import Image, display

with open("eu_zinsen_seit_1993.png", mode='rb') as h:
    image_data = h.read()

display(Image(data=image_data))

In [ ]:
base64_encoded = base64.b64encode(image_data).decode('utf-8')

image_payload = {
  # Das 4o klappt in der Regel besser für Bildanalysen
  "model": "gpt-4o",
  "messages": [
    {"role": "system", 
     "content": "Du bist ein Experte darin PDF Tabellen zu analysieren"
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": """In welchem Jahr war für Griechenland der Zinsatz am höchsten?"""
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/png;base64,{base64_encoded}"
          }
        }         
      ]
    }
  ],
  "temperature": 0.5
}

In [ ]:
endpoint_response = call_openai(payload=image_payload)
endpoint_response.get('choices')[0].get('message', {}).get('content')